In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [64]:
#Path to the main directory
folder_path = '/content/gdrive/My Drive/Colab Notebooks/InnSure/pdl_query/'
profiles_path = folder_path + 'matchProfile_0419.pickle'
profileurls_path = folder_path + 'matchProfileUrl_0419.pickle'

In [3]:
import pandas as pd
import numpy as np
import requests
import json
import time
from datetime import datetime
import pickle
import os
from ast import literal_eval

In [31]:
## loads a pickle/python object
## prints current time
def now(phrase):
    print(f"[{datetime.now()}]: {phrase}")
    
now("Success!")
def load(filename):
    now("starting")
    if (os.path.exists(filename)):
        with open(filename, 'rb') as f:
            data = pickle.load(f)
        now("ending")
        return data
    else:
        raise FileNotFoundError(f"could not find {filename}")


## saves as pickle object
def save(filename, obj):
    now("Starting")
    with open(filename, 'wb') as f:
        pickle.dump(obj, f)
        print(f"Saved obj Object to {filename}!")
    now("Ending")
    return ''

[2022-04-20 13:36:14.517342]: Success!


In [65]:
profiles = load(profiles_path)

[2022-04-20 13:58:56.445662]: starting
[2022-04-20 13:58:56.812695]: ending


In [66]:
len(profiles)

1106

In [67]:
addOthers = ['id','birth_year','search_input_url','gender','industry','inferred_salary','job_onet_broad_occupation',
             'job_onet_major_group','job_onet_broad_occupation','job_onet_minor_group','job_onet_title','job_title',
             'job_title_levels','job_title_role','job_title_sub_role','languages','linkedin_connections', 
             'interests','skills','certifications','search_breach_year','search_input_company']
addYN = ['linkedin_url','facebook_url','twitter_url','github_url']
addLen = ['emails','profiles']

profs = profiles.copy()

newProfiles = [] #list of dictionaries of different profiles
newProfileUrls = [] #list of linkedin links

for profile in profs:
  #profile['data'].keys()
  #print(profile['search_input_url'])
  profile['data']['search_input_url']= profile['search_input_url']
  profile['data']['search_input_company']=profile['search_input_company']
  profile['data']['search_breach_year']=profile['search_breach_year']
  del profile['dataset_version']
  del profile['status']
  del profile['likelihood']
  del profile['search_input_url']
  del profile['search_input_company']
  del profile['search_breach_year']
  profile= profile['data']
  #newProfiles.append(profile)

  cleanProfile = {} #dictionary of each profile, cleaned

  ### KEEP FEATURES  ###
  for i in list(addOthers):
    if i in list(profile):
      cleanProfile[i] = profile[i]

  ### AGGREGATE FEATURES YES/NO ###
  for i in addYN:
    if i in list(profile):
      if profile[i] != None:
        cleanProfile[i] = 1
      else:
        cleanProfile[i] = 0

  ### AGGREGATE FEATURES LENGTH ###
  for i in list(addLen):
    if i in list(profile):
      cleanProfile[i] = len(profile[i])

  ### EXPERIENCE
  cleanProfile['experience']=[]
  if profile['experience'] != None:
    for i in range(len(profile['experience'])):
      job = profile['experience'][i]
      cleanJob = {}    
      toAdd_exp = ['company','end_date','start_date','title','breachJob']
      toAdd_company = ['industry','name','size','type']
      toAdd_title = ['levels','name','role']
      for item1 in toAdd_exp:
        if item1 in list(job):
          if item1 == 'company':
            cleanJob[item1] = {}
            if job[item1] != None:
              companyKeys = list(job[item1])
              for j in toAdd_company:
                if j in companyKeys:
                  cleanJob[item1][j] = job[item1][j]
          elif item1 == 'title':
            cleanJob[item1] = {}
            if job[item1] != None:
              titleKeys = list(job[item1])
              for k in toAdd_title:
                if k in titleKeys:
                  cleanJob[item1][k] = job[item1][k]
          else:
            cleanJob[item1] = job[item1]
      cleanProfile['experience'].append(cleanJob)
  else:
    cleanProfile['experience'].append(profile['experience'])


   ### EDUCATION 
  cleanProfile['education']=[]
  if profile['education'] != None:
    for i in range(len(profile['education'])):
      edu = profile['education'][i]
      cleanEdu = {}
      toAdd_edu = ['degrees','start_date','end_date','majors','type']
      for item1 in toAdd_edu:
        if item1 in list(edu):
          cleanEdu[item1] = edu[item1]
      cleanProfile['education'].append(cleanEdu)
  else:
    cleanProfile['education'].append(profile['education'])


  ### CERTIFICATIONS
  cleanProfile['certifications']=[]
  if profile['certifications'] != None:
    for i in range(len(profile['certifications'])):
      cert = profile['certifications'][i]
      cleanCert = {}
      if 'name' in list(cert):
        #cleanCert['name'] = cert['name']
        cleanProfile['certifications'].append(cert['name'])
  else:
    cleanProfile['certifications'].append(profile['certifications'])

  newProfiles.append(cleanProfile)
  newProfileUrls.append(cleanProfile['search_input_url'])


In [68]:
len(newProfiles)

1106

In [ ]:
'''get text
output_file = open('0420_cleanMatchUrls.txt', 'w')
for element in newProfileUrls: 
  output_file.write(element + '\n')
output_file.close()
'''

In [70]:
with open('matchProfileUrl_0420.pickle', 'wb') as handle:
    pickle.dump(newProfileUrls, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('matchProfileUrl_0420.pickle', 'rb') as handle:
    open_matchProfileUrl = pickle.load(handle)

In [72]:
with open('matchProfile_0420.pickle', 'wb') as handle:
    pickle.dump(newProfiles, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('matchProfile_0420.pickle', 'rb') as handle:
    open_matchProfile = pickle.load(handle)

In [ ]:
df_new_profs = pd.DataFrame(newProfiles)

In [ ]:
df_new_profs.head(5)

,edu0_degrees,edu0_end_date,edu0_start_date,edu0_majors,edu0_school,job0_industry,job0_name,job0_size,job0_type,job0_end_date,...,edu5_majors,edu5_school,job8_industry,job8_name,job8_size,job8_type,job8_end_date,job8_start_date,job8_levels,job8_breachJob
0,"['bachelors', 'bachelor of arts']",1994,1990,[],post-secondary institution,hospital & health care,chief medical information officer,1001-5000,private,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"['doctorates', 'doctor of philosophy']",1977,1975,[],post-secondary institution,hospital & health care,senior programmer and analyst,10001+,private,2014-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,['bachelors'],1995,1991,['biology'],post-secondary institution,hospital & health care,"vice president, isd operations and pop health",10001+,private,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],1997,1995,['medicine'],post-secondary institution,higher education,associate professor anesthesiology and interna...,1001-5000,private,,...,"['health care', 'informatics']",post-secondary institution,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"['doctor of medicine', 'doctorates']",,,[],post-secondary institution,higher education,medical student,1001-5000,private,2008-06-01,...,NaN,NaN,hospital & health care,"director, westside dermatology faculty practice",10001+,private,,2012-08-01,['director'],1
